In [1]:
import os, sys
import argparse
import subprocess
from multiprocessing.pool import Pool
import numpy as np
from collections import OrderedDict
from tqdm import tqdm
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import json
import ast
from skimage import io
from skimage.color import gray2rgb
from tqdm import tqdm
from collections import defaultdict
import neuroglancer

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.file_location import FileLocationManager
from utilities.alignment_utility import (load_consecutive_section_transform,
                                         convert_resolution_string_to_um, SCALING_FACTOR)

viewer = neuroglancer.Viewer()
print(viewer)

Connecting dklab@db.dk.ucsd.edu:3306
http://127.0.0.1:37407/v/2c0b5218ff24d675c5a70335abe666e9097809f4/


In [2]:
animal = 'MD594'
resolution = 'thumbnail'
aligned_shape = np.array((47000, 23300)) // 32
num_section = 433

color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}

## Get the annotation points

In [3]:
def get_dense_coordinates(coor_list):
    dense_coor_list = []
    # Shortest distance, x, y

    # for x, y in coor_list:
    for i in range(len(coor_list) - 1):
        x, y = coor_list[i]
        x_next, y_next = coor_list[i + 1]

        x_mid = (x + x_next) / 2
        y_mid = (y + y_next) / 2

        dense_coor_list.append([x, y])
        dense_coor_list.append([x_mid, y_mid])

        if i == len(coor_list) - 2:
            dense_coor_list.append([x_next, y_next])
            x, y = coor_list[0]
            x_mid = (x + x_next) / 2
            y_mid = (y + y_next) / 2
            dense_coor_list.append([x_mid, y_mid])

    return dense_coor_list

def get_contours_from_annotations(stack, target_structure, hand_annotations, densify=0):
    MD585_ng_section_min = 83
    num_annotations = len(hand_annotations)
    str_contours_annotation = {}

    for i in range(num_annotations):
        structure = hand_annotations['name'][i]
        side = hand_annotations['side'][i]
        section = hand_annotations['section'][i]
        first_sec = 0
        last_sec = 0

        #if side == 'R' or side == 'L':
        #    structure = structure + '_' + side

        if structure == target_structure:
            vertices = hand_annotations['vertices'][i]

            for i in range(densify):
                vertices = get_dense_coordinates(vertices)

            # Skip sections before the 22nd prep2 section for MD585 as there are clear errors
            if stack == 'MD585' and section < MD585_ng_section_min + 22:
                # vertices = vertices - np.array(MD585_abberation_correction)
                continue
            str_contours_annotation[section] = {}
            str_contours_annotation[section][structure] = {}
            str_contours_annotation[section][structure][1] = vertices

    try:
        first_sec = np.min(list(str_contours_annotation.keys()))
        last_sec = np.max(list(str_contours_annotation.keys()))
    except:
        pass
    return str_contours_annotation, first_sec, last_sec

CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
csvfile = os.path.join(CSV_PATH, f'{animal}_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))
hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))

structures = list(hand_annotations['name'].unique())
section_structure_vertices = defaultdict(dict)
for structure in tqdm(structures):
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations, densify=4)
    for section in contour_annotations:
        section_structure_vertices[section][structure] = contour_annotations[section][structure][1]

100%|██████████| 65/65 [00:06<00:00,  9.34it/s]


## Reproduce create_clean transform

In [4]:
PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1'
thumbnail_dir = os.path.join(PATH, 'thumbnail')

section_offset = {}
for file_name in tqdm(sorted(os.listdir(thumbnail_dir))):
    filepath = os.path.join(thumbnail_dir, file_name)
    img = io.imread(filepath)
    section = int(file_name.split('.')[0])
    section_offset[section] = (aligned_shape - img.shape[:2][::-1]) // 2

  0%|          | 0/433 [00:00<?, ?it/s]


ValueError: TiffPage 0: <COMPRESSION.LZW: 5> requires the 'imagecodecs' package

## Reproduce create_alignment transform

In [5]:
def parse_elastix(animal):
    """
    After the elastix job is done, this goes into each subdirectory and parses the Transformation.0.txt file
    Args:
        animal: the animal
    Returns: a dictionary of key=filename, value = coordinates
    """
    fileLocationManager = FileLocationManager(animal)
    DIR = fileLocationManager.prep
    INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')

    image_name_list = sorted(os.listdir(INPUT))
    anchor_idx = len(image_name_list) // 2
    # anchor_idx = len(image_name_list) - 1
    transformation_to_previous_sec = {}

    for i in range(1, len(image_name_list)):
        fixed_fn = os.path.splitext(image_name_list[i - 1])[0]
        moving_fn = os.path.splitext(image_name_list[i])[0]
        transformation_to_previous_sec[i] = load_consecutive_section_transform(animal, moving_fn, fixed_fn)

    transformation_to_anchor_sec = {}
    # Converts every transformation
    for moving_idx in range(len(image_name_list)):
        if moving_idx == anchor_idx:
            transformation_to_anchor_sec[image_name_list[moving_idx]] = np.eye(3)
        elif moving_idx < anchor_idx:
            T_composed = np.eye(3)
            for i in range(anchor_idx, moving_idx, -1):
                T_composed = np.dot(np.linalg.inv(transformation_to_previous_sec[i]), T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed
        else:
            T_composed = np.eye(3)
            for i in range(anchor_idx + 1, moving_idx + 1):
                T_composed = np.dot(transformation_to_previous_sec[i], T_composed)
            transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed


    return transformation_to_anchor_sec

def convert_2d_transform_forms(arr):
    return np.vstack([arr, [0,0,1]])

def create_warp_transforms(animal, transforms, transforms_resol, resolution):
    #transforms_resol = op['resolution']
    transforms_scale_factor = convert_resolution_string_to_um(animal, resolution=transforms_resol) / convert_resolution_string_to_um(animal, resolution=resolution)
    tf_mat_mult_factor = np.array([[1, 1, transforms_scale_factor], [1, 1, transforms_scale_factor]])
    transforms_to_anchor = {
        img_name:
            convert_2d_transform_forms(np.reshape(tf, (3, 3))[:2] * tf_mat_mult_factor) for
        img_name, tf in transforms.items()}

    return transforms_to_anchor

transforms = parse_elastix(animal)
warp_transforms = create_warp_transforms(animal, transforms, 'thumbnail', resolution)
ordered_transforms = sorted(warp_transforms.items())

section_transform = {}
for section, transform in ordered_transforms:
    section_num = int(section.split('.')[0])
    transform = np.linalg.inv(transform)
    section_transform[section_num] = transform

## Alignment of annotation coordinates

In [24]:
'''
(x', y') = (x * sx + y * ry + tx, x * rx + y * sy + ty)
'sx': T[0, 0], 'sy': T[1, 1], 'rx': T[1, 0], 'ry': T[0, 1], 'tx': T[0, 2], 'ty': T[1, 2]
'''
def transform_create_alignment(points, transform):
    a = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.T[:, 0:2]
    c = np.matmul(a, b)
    return c

volume = np.zeros((aligned_shape[1], aligned_shape[0], num_section), dtype=np.uint8)
for section in section_structure_vertices:
    template = np.zeros((aligned_shape[1], aligned_shape[0]), dtype=np.uint8)
    for structure in section_structure_vertices[section]:
        points = np.array(section_structure_vertices[section][structure])
        points = points // 32
        points = points + section_offset[section] # create_clean offset
        points = transform_create_alignment(points, section_transform[section]) # create_alignment transform
        points = points.astype(np.int32)
        
        try:
            color = colors[structure.upper()]
        except:
            sided = '{}_R'.format(structure)
            try:
                color = colors[sided]
            except:
                color = 255
            
        cv2.polylines(template, [points], True, color, 10, lineType=cv2.LINE_AA)
    volume[:, :, section - 1] = template

In [26]:
volume = np.swapaxes(volume, 0, 1)
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[14464, 14464, 20000]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [ ]:
with open(f'{animal}_annotations.npy', 'wb') as file:
    np.save(volume, file)